In [101]:
import numpy as np
import pandas as Pd

In [102]:
class Vasicek:

    def __init__(self,kappa,theta,sigma):
        self.kappa=kappa
        self.theta=theta
        self.sigma=sigma

In [103]:
hw3dynamics = Vasicek(kappa=3,theta=0.05,sigma=0.03)

In [104]:
class Bond:

    def __init__(self, T):
        self.T=T


In [105]:
hw3contract = Bond(T=5)

In [106]:
class FDexplicitEngine:

    def __init__(self, rMax, rMin, deltar, deltat, useUpwind):
        self.rMax=rMax
        self.rMin=rMin
        self.deltar=deltar
        self.deltat=deltat
        self.useUpwind=useUpwind

    def price_bond_vasicek(self,contract,dynamics):
    # You complete the coding of this function
    #
    # Returns array of all initial short rates,
    # and the corresponding array of zero-coupon
    # T-maturity bond prices
        deltax=self.deltar
        deltat=self.deltat
        T = contract.T
        N=round(T/self.deltat)
        if abs(N-T/self.deltat) > 1e-12:
            raise ValueError("Bad delta t")

        r=np.arange(self.rMax,self.rMin-self.deltar/2,-self.deltar)   
        #I'm making the FIRST indices of the array correspond to HIGH levels of r
        bondprice=np.ones(np.size(r))

        if self.useUpwind: # special useUpwind calculation 
            nu = dynamics.kappa * (dynamics.theta - r)
            param_1 = (np.power(dynamics.sigma, 2) * self.deltat) / (2 * np.power(self.deltar, 2))
            param_2 = deltat / deltax * nu
            qu = np.where(nu >= 0, param_1 + param_2, param_1)
            qd = np.where(nu >= 0, param_1, param_1 - param_2)
            qm = 1 - qu - qd
            
        else: # normal calculation way
            
            nu = dynamics.kappa * (dynamics.theta - r)
            param_1 = (np.power(dynamics.sigma, 2) * self.deltat) / (np.power(self.deltar, 2))
            qu = 0.5*(param_1 + nu*deltat/deltax)
            qd = 0.5*(param_1 - nu*deltat/deltax)
            qm = 1 - qd - qu
            

        for t in np.arange(N-1,-1,-1)*self.deltat:
            bondprice=1/(1+r*self.deltat)*(qd*np.roll(bondprice,-1)+qm*bondprice+qu*np.roll(bondprice,1))

            # It is not obvious in this case,
            # what boundary conditions to use at the top and bottom
            # so let us assume "linearity" boundary conditions
            bondprice[0]=2*bondprice[1]-bondprice[2]
            bondprice[-1]=2*bondprice[-2]-bondprice[-3]

        return (r, bondprice,qu,qd,param_1)

Q1.a

$\ dC = C'_r \ dr + C'_t \  dt + 0.5 C''_{rr} \ d^2r$



plug in $\ dr = a(r_t, t) \ dt + b(r_t, t) \ dW_t$



and the drift term of $\ dC$ is rC



$\frac{\partial C}{\partial r_t} a(r_t, t) + \frac{1}{2} \frac{\partial^2 C}{\partial r_t^2} b^2(r_t, t) + \frac{\partial C}{\partial t} = rC$


#Q1.d
see the pdf file


In [107]:
#Q1.e central difference
hw3FD = FDexplicitEngine(rMax=0.35, rMin=-0.25, deltar=0.01, deltat=0.01, useUpwind=False)
(r, bondprice,qu,qd,para_1) = hw3FD.price_bond_vasicek(hw3contract, hw3dynamics)
np.set_printoptions(precision=4, suppress=True)
displayrows = (r < 0.15 + hw3FD.deltar / 2) & (r > 0.0 - hw3FD.deltar / 2)
print(np.stack((r, bondprice), axis=1)[displayrows])

[[ 1.5000e-01 -1.4273e+09]
 [ 1.4000e-01  1.6361e+08]
 [ 1.3000e-01  2.2294e+07]
 [ 1.2000e-01 -1.3724e+06]
 [ 1.1000e-01 -1.3361e+05]
 [ 1.0000e-01  3.2966e+03]
 [ 9.0000e-02  1.3021e+02]
 [ 8.0000e-02  7.7128e-01]
 [ 7.0000e-02  7.7385e-01]
 [ 6.0000e-02  7.7643e-01]
 [ 5.0000e-02  7.7902e-01]
 [ 4.0000e-02  7.8162e-01]
 [ 3.0000e-02  7.8423e-01]
 [ 2.0000e-02  7.8685e-01]
 [ 1.0000e-02  1.4165e+03]
 [-3.3307e-16  5.1498e+04]]


In [111]:
#use Upwind
hw3FD = FDexplicitEngine(rMax=0.35, rMin=-0.25, deltar=0.01, deltat=0.01, useUpwind=True)
(r, bondprice,qu,qd,para_1) = hw3FD.price_bond_vasicek(hw3contract, hw3dynamics)
np.set_printoptions(precision=4, suppress=True)
displayrows = (r < 0.15 + hw3FD.deltar / 2) & (r > 0.0 - hw3FD.deltar / 2)
print(np.stack((r, bondprice), axis=1)[displayrows])

[[ 0.15    0.7536]
 [ 0.14    0.7561]
 [ 0.13    0.7586]
 [ 0.12    0.7611]
 [ 0.11    0.7637]
 [ 0.1     0.7662]
 [ 0.09    0.7688]
 [ 0.08    0.7713]
 [ 0.07    0.7739]
 [ 0.06    0.7765]
 [ 0.05    0.7791]
 [ 0.04    0.7817]
 [ 0.03    0.7843]
 [ 0.02    0.7869]
 [ 0.01    0.7895]
 [-0.      0.7922]]


As a rsult, the calculation using upwind method is more accurate

Q1.f
Ignoring stability issues and considering only consistency (i.e. “truncation error,”
also known as “local discretization error”), the upwind explicit scheme, which uses
one-sided spatial differences, discretizes the PDE with ***less*** accuracy than the
standard explicit scheme, which uses central spatial differences.
However, to actually guarantee convergence, the grid spacing must satisfy certain
stability constraints. In a PDE exhibiting strong drift, we have just seen that these
constraints may allow the upwind scheme ***more*** freedom in choosing grid spacing,
compared to the standard scheme.

In [117]:
#Q1.g
#r=0.12
display(np.log(1/0.7611) / 5)

0.054598104741758226

In [118]:
#r=0.02
display(np.log(1/0.7869) / 5)

0.04793082068823988

when r=0.12 then drift become nagetive. then the YTM going down. However, when r =0.02 the drift is positive so the YTM is bigger than 0.02

Q2.a

delta = N(d1)

d1 = (log(S_0/K) + (r+sigma^2/2) * T)/ (sigma * sqrt(T))

so as a result delta = N^{-1}(d1) which indicates we could have an equation K = f(S_0, delta, sigma, T, r)

In [132]:
#Q2.b

from scipy.stats import norm
import numpy as np

delta = 0.25
S0 = 300
T = 1/12
sigma = 0.4
r = 0.01
N_inv_delta = norm.ppf(delta)

K_25 = S0 * np.exp(-N_inv_delta * sigma * np.sqrt(T) + (r - 0.5 * sigma**2) * T)
K_25    

322.4127345545695

In [133]:
d1 = (np.log(S0 / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
d2 = d1 - sigma * np.sqrt(T)
call_price_25 = (S0 * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2))
call_price_25

28.80903631539175

In [134]:
delta = 0.75
N_inv_delta = norm.ppf(delta)
K_75 = S0 * np.exp(-N_inv_delta * sigma * np.sqrt(T) + (r - 0.5 * sigma**2) * T)
K_75    

275.90753005697735

In [135]:
d1 = (np.log(S0 / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
d2 = d1 - sigma * np.sqrt(T)
call_price_75 = (S0 * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2))
call_price_75

28.80903631539175

In [137]:
#Q2.c
lambda_25=0.25*S0/call_price_25
lambda_25

2.6033498371457116

In [138]:
lambda_75=0.75*S0/call_price_75
lambda_75

7.810049511437134

From the result above we could see the delta = 0.75 which leads to higher lambda. As a result ITM call lambda is higher.